In [1]:
from models import *
from data import get_dataloaders
import torch
from importlib import reload

LOAD_EXPERIMENT = "fixslice-sbox-byte1-zhang-0_1000"
LOAD_EPOCH = 65

device = torch.device("cuda")

sbox_model = torch.load(f"models/{LOAD_EXPERIMENT}/epoch{LOAD_EPOCH}.pt")
sbox_model.to(device)

_, _, test_loader = get_dataloaders(
    200,
    "sbox",
    1,
    0,
    1000,
)


C:\Users\Ulrik\AppData\Local\Temp\ipykernel_10984\1364221038.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sbox_model = torch.load(f"models/{LOAD_EXPERIMENT}/epoch{LO

In [16]:
from data import s_box, inverted_s_box

In [17]:
_, plaintext, _ = test_loader.__iter__().__next__()
plaintext = plaintext[:,:,0].squeeze()

In [18]:
sk_permutations = torch.zeros([500,256])
for pt_idx, pt in enumerate(plaintext.long()):
    for idx in torch.arange(0,256):
        sk_permutations[pt_idx, idx] = inverted_s_box[idx] ^ pt

sk_permutations = sk_permutations.long()

In [19]:
import keyrank_rs
keyrank_rs = reload(keyrank_rs)

rs_sk_perm = torch.Tensor(keyrank_rs.sbox_key_permutations(plaintext.long().tolist())).long()

In [20]:
model_output = torch.rand([500, 256])

permuted = model_output.gather(1, sk_permutations.long())

In [21]:
trace, plaintext, key = test_loader.__iter__().__next__()

print(trace.shape)
print(plaintext.shape)
print(key.shape)

torch.Size([1, 500, 1000])
torch.Size([1, 500, 2])
torch.Size([1])


In [22]:
sbox_scores = sbox_model(trace.squeeze().to(device))
print(sbox_scores.type())

torch.cuda.FloatTensor


In [23]:
sbox_scores = torch.arange(0, 256).repeat([500, 1]).float().to(device)

In [24]:
mapping_plaintexts = plaintext[..., 0].squeeze().long().detach().cpu().numpy()
np_sbox_scores = sbox_scores.detach().cpu().numpy()

rust = keyrank_rs.sbox_scores_to_keyscores_parallel(mapping_plaintexts, np_sbox_scores)

gathered = sbox_scores.gather(1, rs_sk_perm.to(device))

In [14]:
from_rust = torch.Tensor(rust).to(device)

(from_rust == gathered).sum()

tensor(128000, device='cuda:0')

In [28]:
from tqdm import tqdm

In [29]:
for trace, plaintexts, key in tqdm(test_loader, unit='key'):
    sbox_scores = sbox_model(trace.squeeze().to(device))

    mapping_plaintexts = plaintext[..., 0].squeeze().long().detach().cpu().numpy()
    np_sbox_scores = sbox_scores.detach().cpu().numpy()

    rust = keyrank_rs.sbox_scores_to_keyscores_parallel(mapping_plaintexts, np_sbox_scores)



100%|██████████| 500/500 [00:21<00:00, 23.11key/s]


In [31]:
for trace, plaintexts, key in tqdm(test_loader, unit='key'):
    sbox_scores = sbox_model(trace.squeeze().to(device))

    rs_sk_perm = torch.Tensor(keyrank_rs.sbox_key_permutations(plaintexts[...,0].squeeze().long().tolist())).long()

    gathered = sbox_scores.gather(1, rs_sk_perm.to(device))

100%|██████████| 500/500 [00:20<00:00, 24.00key/s]
